In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from data_analysis_config import col_name_map, TYPES_COLUMN_NAMES, categorize_by_quantile
from itertools import chain
import glob
import os


## **PRE-WORK**
### PREPARE DATA
1. Load data
2. Rename columns
3. Delete non-numeric columns


In [103]:
# load dataframe
key_df = pd.read_csv('../csv_output/key_misspells.csv', sep=';')
fp_df = pd.read_csv('../csv_output/fp.csv', sep=';')
rand_df = pd.read_csv('../csv_output/rand_misspells.csv', sep=';')

fp_df['amount_of_errors'] = 0

# rename dataframe
fp_df.rename(columns=col_name_map, inplace=True)
key_df.rename(columns=col_name_map, inplace=True)
rand_df.rename(columns=col_name_map, inplace=True)

fp_df['Function'] = fp_df["Function"].astype("category")
key_df['Function'] = key_df["Function"].astype("category")
rand_df['Function'] = rand_df["Function"].astype("category")

# delete strings
fp_df = fp_df.select_dtypes(exclude=['object'])
key_df = key_df.select_dtypes(exclude=['object'])
rand_df = rand_df.select_dtypes(exclude=['object'])

fp_df['Function'] = fp_df["Function"].astype("object")
key_df['Function'] = key_df["Function"].astype("object")
rand_df['Function'] = rand_df["Function"].astype("object")

In [104]:
columns = ['Damerau–Levenshtein normalized similarity'] + [col for cat in TYPES_COLUMN_NAMES.values() for col in cat] + ['Function', 'Amount of misspells (in input sentence)']
fp_df = fp_df[columns]
key_df = key_df[columns]
rand_df = rand_df[columns]

### Add each sentence to a bucket: 
1. [1, 3) errors
2. [3, 6) errors
3. [6, +inf)

In [105]:
key_df['Bucket'] = key_df['Amount of misspells (in input sentence)'].apply(lambda x: 1 if x < 3 else ( 2 if 3 <= x <= 5 else 3))
rand_df['Bucket'] = key_df['Amount of misspells (in input sentence)'].apply(lambda x: 1 if x < 3 else ( 2 if 3 <= x <= 5 else 3))

In [106]:
key_based_dataframes = [key_df.loc[key_df['Bucket'] == i].reset_index(drop=True) for i in range(1, 4)]
rand_based_dataframe = [rand_df.loc[rand_df['Bucket'] == i].reset_index(drop=True)  for i in range(1, 4)]
del key_df, rand_df


#### Define quantiles and categorize each of metrics (basing on perfect match & quantiles)

In [107]:
config = pd.DataFrame()
CATEGORIES = [1,2,3,4] # WHERE 1 IS TOP and 4 IS BOTTOM


for by in list(chain.from_iterable(TYPES_COLUMN_NAMES.values())):
    # define the quantiles
    quantiles = {"key": [], 'rand': [], 'fp': []}
    for i in range(len(key_based_dataframes)):
        quantiles['key'].append(key_based_dataframes[i][by].quantile([0.25, 0.5, 0.75]).values)
        quantiles['rand'].append(rand_based_dataframe[i][by].quantile([0.25, 0.5, 0.75]).values)
    quantiles['fp'] = (fp_df[by].quantile([0.25, 0.5, 0.75]).values)


    if 'normalized similarity'.lower() in by.lower():
        ascending = True
        goal = 1.0
    else: 
        ascending = False
        goal = 0.0


    for i in range(len(quantiles['key'])):
        config[f'{by} key group {i} bucket'] = np.concatenate((quantiles['key'][i], [goal, ascending]))
        config[f'{by} rand group {i} bucket'] = np.concatenate(
            (quantiles['rand'][i], [goal, ascending])
        )
    # assign to config quantiles, goal and ascending    
    config[f'{by} fp quantiles'] = np.concatenate((quantiles['fp'], [goal, ascending]))

    for k in range(len(key_based_dataframes)):
        key_based_dataframes[k].insert(list(key_based_dataframes[k].columns).index(by), by + " Result category" , key_based_dataframes[k][by].apply(lambda x: categorize_by_quantile(x, goal, quantiles['key'][k], values = CATEGORIES, ascending_is_better=ascending)))
        rand_based_dataframe[k].insert(list(rand_based_dataframe[k].columns).index(by), by + " Result category" , rand_based_dataframe[k][by].apply(lambda x: categorize_by_quantile(x, goal, quantiles['rand'][k], values =  CATEGORIES, ascending_is_better=ascending)))
    fp_df.insert(list(fp_df.columns).index(by), by + " Result category" , fp_df[by].apply(lambda x: categorize_by_quantile(x, goal, quantiles['fp'], values =  CATEGORIES, ascending_is_better=ascending)))


In [108]:
config = config.transpose()


In [109]:
config.columns
names = {0: 'quantile 1', 1: 'quantile 2', 2: 'quantile 3', 3: 'Goal', 4: 'Ascending'}
config.rename(names, inplace=True, axis=1)


#### Define CONSTS

In [110]:
COLUMNS_FOR_RESULT_CATEGORY = [col for col in fp_df.columns if 'Result category' in col]
COLUMN_NAMES_FOR_CATEGORIES = ['Full correctness', 'First range', 'Second range', 'Third range']

#### Filter columns

In [111]:
for k in range(len(key_based_dataframes)):
    key_based_dataframes[k] = key_based_dataframes[k][COLUMNS_FOR_RESULT_CATEGORY + ['Function']]
    rand_based_dataframe[k] = rand_based_dataframe[k][COLUMNS_FOR_RESULT_CATEGORY + ['Function']]
fp_df = fp_df[COLUMNS_FOR_RESULT_CATEGORY + ['Function']]


In [112]:
def get_summary(data_frame, column_names_for_values: list, category_values_to_parse: list, normalized_values: bool = False):
    if normalized_values:
        column_names_for_values = [str(x) + ' [%]' for x in column_names_for_values]
    scores = pd.DataFrame()
    for by in data_frame.columns[:-1]:
        # Compute the cross-tabulation of the 'Function' and the column
        scores = pd.concat([scores, data_frame.groupby('Function')[by].agg(
            [(f' {by} ({column_names_for_values[0].lower()})', lambda x: (x == category_values_to_parse[0]).sum()),
             (f' {by} ({column_names_for_values[1].lower()})', lambda x: (x == category_values_to_parse[1]).sum()),
             (f' {by} ({column_names_for_values[2].lower()})', lambda x: (x == category_values_to_parse[2]).sum()),
             (f' {by} ({column_names_for_values[3].lower()})', lambda x: (x == category_values_to_parse[3]).sum()),

             ])], axis=1)

    scores = pd.concat([scores, data_frame.groupby('Function')[data_frame.columns[0]].agg(
        [('Total count', 'count')])], axis=1)
    if normalized_values:
        for col_name in scores.columns[:-1]:
            scores[col_name] = scores[col_name] / scores['Total count']
    return scores
    

In [113]:
def get_weitghted_average (dataframe, weights, only_average_columns=False):
    for i in COLUMNS_FOR_RESULT_CATEGORY:
        summary_column_names = [k for k in dataframe.columns if i in k]
        dataframe[f'Weighted average {i}'] = dataframe[summary_column_names].dot(weights)
    averages_columns = [k for k in dataframe.columns if "Weighted average" in k]

    if only_average_columns:
        return dataframe[averages_columns], averages_columns
    return dataframe, averages_columns


## **ANALYSIS**

In [114]:
summary = {}
QUANTIL_RANGE_WEIGTHS = {'fp': [1,0,0,0], 'key': [1,0,0,0], 'rand': [1,0,0,0]}
# QUANTIL_RANGE_WEIGTHS = {'fp': [1,0,0,-0.5], 'key': [1,0,0,-0.5], 'rand': [1,0,0,-0.5]}
                                                     # weigths for perfect match  & quantiles 
# fp 
summary_df, averages_columns = get_weitghted_average(get_summary(fp_df, column_names_for_values=COLUMN_NAMES_FOR_CATEGORIES, category_values_to_parse=CATEGORIES, normalized_values=True), QUANTIL_RANGE_WEIGTHS['fp'])
summary_df['Summary'] = summary_df[averages_columns].sum(axis=1)
summary['FP'] = summary_df
for k in range(len(key_based_dataframes)):
    # key based
    summary_df, averages_columns = get_weitghted_average(get_summary(key_based_dataframes[k], column_names_for_values=COLUMN_NAMES_FOR_CATEGORIES, category_values_to_parse=CATEGORIES, normalized_values=True), QUANTIL_RANGE_WEIGTHS['key'])
    summary_df['Summary'] = summary_df[averages_columns].sum(axis=1)
    summary[f'key {k+1}'] = summary_df
    # random
    summary_df, averages_columns = get_weitghted_average(get_summary(rand_based_dataframe[k], column_names_for_values=COLUMN_NAMES_FOR_CATEGORIES, category_values_to_parse=CATEGORIES, normalized_values=True), QUANTIL_RANGE_WEIGTHS['rand'])
    summary_df['Summary'] = summary_df[averages_columns].sum(axis=1)
    summary[f'random {k+1}'] = summary_df
    

In [115]:
# Concatenate the columns from each dataframe into a single dataframe
merged_column_df = pd.concat([df["Summary"] for df in summary.values()], axis=1,keys=summary.keys())

# Set the index of the merged dataframe to the index of one of the source dataframes
index_df = list(summary.values())[0]  # choose one of the source dataframes as index reference
merged_column_df = merged_column_df.set_index(index_df.index)


In [116]:
# POINTS = [30, 25, 22, 18, 15] + list(range(10, 0, -1))
POINTS = [i for i in range(5, 0, -1)]

total = pd.DataFrame(0, index=merged_column_df.index, columns=merged_column_df.columns)
for col in merged_column_df.columns:
    sorted_df = merged_column_df[col].sort_values(ascending=False)
    for i, tool in enumerate(sorted_df.index[:5]):
        total.loc[tool, col] = POINTS[i]


In [117]:
ERROR_TYPE_CATEGORY_WEIGTH = [1,1,1,1,1,1,1]
# ERROR_TYPE_CATEGORY_WEIGTH = [0.8, 1, 1, 0.8, 0.8, 0.5, 0.5]
total['TOTAL'] = np.dot(total, ERROR_TYPE_CATEGORY_WEIGTH)
total.sort_values(by='TOTAL', inplace=True, ascending=False)


In [118]:
consts = {
"POINTS FOR TOP N": tuple(POINTS) , 
"ERROR_TYPE_CATEGORY_WEIGTH (FP, [KEY(n), RAND(n)] for n in range(3))": tuple(ERROR_TYPE_CATEGORY_WEIGTH), 
"PERFECT MATCH, 1-4 QUANTILES RESUULT WEIGHT": tuple(QUANTIL_RANGE_WEIGTHS)
}
consts = pd.DataFrame.from_dict(consts, orient='index').transpose()

In [119]:
title = "".join(x for x in str(ERROR_TYPE_CATEGORY_WEIGTH) if (x.isalnum() or x in "._"))  + '_p_' + ''.join(x for x in str(POINTS) if (x.isalnum() or x in "._-")) + '_w_' + "".join(x for x in str(list(QUANTIL_RANGE_WEIGTHS.values())) if (x.isalnum() or x in "._-")).replace('.', '_') 

with pd.ExcelWriter(f'./excel_files/P2/{title}.xlsx') as writer:
    consts.to_excel(writer, sheet_name='consts_config', index=False)
    config.to_excel(writer, sheet_name='config', index=True)
    for k in summary.keys():
        summary[k].to_excel(writer, sheet_name=f'{k}', index=True)
    merged_column_df.to_excel(writer, sheet_name='merged_columns', index=True)
    total.to_excel(writer, sheet_name='total', index=True)

In [120]:
from itertools import combinations

def check_occurences_in_excel_files(filepaths, sheet_name, row_counts):
    files = [pd.read_excel(fp, sheet_name=sheet_name, index_col=0) for fp in filepaths]
    file_combinations = list(combinations(range(len(filepaths)), 2))
    
    result = {
        "pairwise": {f"files_{i}_{j}": {} for i, j in file_combinations},
        "all_files": {},
    }
    
    for row_count in row_counts:
        # Pairwise comparison
        for i, j in file_combinations:
            common_indices = set(files[i].index[:row_count]).intersection(files[j].index[:row_count])
            result["pairwise"][f"files_{i}_{j}"][row_count] = common_indices
        
        # Comparison across all files
        common_indices = set(files[0].index[:row_count])
        for file in files[1:]:
            common_indices = common_indices.intersection(file.index[:row_count])
        result["all_files"][row_count] = common_indices

    return result

In [121]:
filepaths = glob.glob(os.path.join('./excel_files', '*.xlsx'))
sheet_name = 'total'
row_counts = [5, 10, 15, 20, 25]
print(filepaths)

comparison_results = check_occurences_in_excel_files(filepaths, sheet_name, row_counts)
for i in comparison_results['all_files'].keys():
    print(i, len(comparison_results['all_files'][i]))

comparison_results['all_files'][20]



[]


IndexError: list index out of range

In [ ]:
from collections import defaultdict
def assign_values_for_occurences_in_excel_files(filepaths, sheet_name, row_counts):
    files = [pd.read_excel(fp, sheet_name=sheet_name, index_col=0) for fp in filepaths]

    result = {}

    for row_count in row_counts:
        index_sums = defaultdict(int)

        for file in files:
            for idx, index_label in enumerate(file.index[:row_count]):
                index_sums[index_label] += idx + 1  # Add 1 to account for 0-based indexing

        result[row_count] = dict(index_sums)

    return result



In [ ]:
filepaths = glob.glob(os.path.join('./excel_files', '*.xlsx'))
sheet_name = 'total'
row_counts = [5, 10, 15, 20, 25]
print(filepaths)

comparison_results = assign_values_for_occurences_in_excel_files(filepaths, sheet_name, row_counts)
# for i in comparison_results['all_files'].keys():
#     print(i, len(comparison_results['all_files'][i]))

# comparison_results['all_files'][25]
for v in comparison_results.values():
    print(dict(sorted(v.items(), key=lambda item: item[1])))


['./excel_files/0.8110.80.80.50.5_p_302522181512151413121110987654321_w_100-0_5100-0_5100-0_5.xlsx', './excel_files/0.8110.80.80.50.5_p_151413121110987654321_w_100-0_5100-0_5100-0_5.xlsx', './excel_files/0.8110.80.80.50.5_p_151413121110987654321_w_100010001000.xlsx', './excel_files/0.8110.80.80.50.5_p_302522181512151413121110987654321_w_100010001000.xlsx']
{'SpellChecker(Gingerit)': 4, 'SpellChecker(LanguageTool)': 5, 'SpellChecker(GramFormer)': 6, 'Autocorrect(GramFormer)': 8, 'GramFormer(Autocorrect)': 10, 'GramFormer(Gingerit)': 13, 'Gingerit(GramFormer)': 14}
{'SpellChecker(Gingerit)': 4, 'Autocorrect(GramFormer)': 8, 'Autocorrect': 10, 'Gingerit': 10, 'LanguageTool(GramFormer)': 10, 'Gingerit(GramFormer)': 14, 'Autocorrect(SpellChecker)': 15, 'GramFormer(LanguageTool)': 16, 'Gingerit(SpellChecker)': 17, 'GramFormer(Gingerit)': 19, 'SpellChecker(GramFormer)': 21, 'SpellChecker(LanguageTool)': 23, 'GramFormer(Autocorrect)': 25, 'Gingerit(LanguageTool)': 28}
{'SpellChecker(Gingerit)'

In [ ]:
# sns.set(rc={'figure.figsize':(16,6)})
# fig, axs = plt.subplots(ncols=3)
# grouped_df = scores.groupby('bucket')
# for i, col in enumerate(grouped_df.columns):
#     sns.barplot(data=grouped_df, x=grouped_df.index, y=col, ax=axs[i], palette='viridis')
#     axs[i].set_xlabel('Groups')
#     axs[i].set_ylabel('Data')
#     axs[i].set_title(col)


# # scores[['Full correctness',	'First range',	'Second range']].plot(kind='bar', stacked = True)
# # plt.set_xlabel('Function')
# # plt.bar(scores.index, scores['Full correctness'], label='Full correctness')
# # plt.bar(scores.index, scores['Full correctness'], bottom=scores['First range'], label='Second range')

# sns.barplot(data=scores, x=scores.index, y=COLUMN_NAMES_FOR_CATEGORIES[0], color='blue', label=by)
# sns.barplot(data=scores, x=scores.index, y=COLUMN_NAMES_FOR_CATEGORIES[1], color='green', bottom=scores[COLUMN_NAMES_FOR_CATEGORIES[0]], label=by)
# sns.barplot(data=scores, x=scores.index, y=COLUMN_NAMES_FOR_CATEGORIES[2], color='red', bottom=scores[COLUMN_NAMES_FOR_CATEGORIES[0]] + scores[COLUMN_NAMES_FOR_CATEGORIES[1]], label=by)

# # Set labels and legend
# plt.xticks(rotation=90)
# plt.rcParams["axes.labelsize"] = 7
# plt.xlabel('Groups')
# plt.ylabel('Data')
# plt.title('Stacked Bar Plot')
# plt.legend()

# # Show plot
# plt.show()


# # plt.bar =

# ### potrojny 